In [ ]:
# Borrar contenido de carpeta midis
!rm -rf midis/*

In [7]:
# Crear carpeta midis
!mkdir midis
# Insertar en ella los archivos .mid

In [2]:
!pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.1 MB/s eta 0:00:00


#Num instrumentos según tracks

In [8]:
import glob
import mido

# Listar todos los archivos .mid en la carpeta "midis"
midi_files = glob.glob("midis/*.mid")

# Recorrer la lista de archivos, contar pistas y buscar drums
for midi_file in midi_files:
    try:
        # Cargar el archivo MIDI usando mido
        mid = mido.MidiFile(midi_file)

        # Contar el número de instrumentos
        num_tracks = len(mid.tracks)

        #check that name doesn't contain "modified"
        if num_tracks > 9 and "modified" not in midi_file:
          print(f"El archivo '{midi_file}' contiene más de 9 pistas")

    except Exception as e:
        print(f"No se pudo procesar el archivo '{midi_file}': {e}")

El archivo 'midis/ST Unknown 5.mid' contiene más de 9 pistas
El archivo 'midis/SS Ballad of the Goddess.mid' contiene más de 9 pistas


#Según program_change

##Núm instrumentos > 9

In [9]:
import glob
import mido

# Listar todos los archivos .mid en la carpeta "midis"
midi_files = glob.glob("midis/*.mid")

for midi_file in midi_files:
    midi = mido.MidiFile(midi_file)
    instrumentos_por_canal = {}

    for track in midi.tracks:
        for msg in track:
            if msg.type == 'program_change':
                canal = msg.channel
                programa = msg.program  # 0-127, General MIDI
                if canal not in instrumentos_por_canal:
                    instrumentos_por_canal[canal] = set()
                instrumentos_por_canal[canal].add(programa)

    # Contar todos los instrumentos únicos
    instrumentos_unicos = set()
    for programas in instrumentos_por_canal.values():
        instrumentos_unicos.update(programas)

    nPistas = len(instrumentos_unicos)
    if nPistas > 9 and "modified" not in midi_file:
    #if nPistas > 9:
        print(f"El archivo '{midi_file}' contiene más de 9 instrumentos: {nPistas} instrumentos")


El archivo 'midis/MC Mt. Crenel.mid' contiene más de 9 instrumentos: 11 instrumentos
El archivo 'midis/MC Staff Roll.mid' contiene más de 9 instrumentos: 33 instrumentos
El archivo 'midis/MC Cloud Tops.mid' contiene más de 9 instrumentos: 21 instrumentos


##Núm de archivos no aptos para MusicVAE

En este bloque se guardan los ficheros aptos para el siguiente

In [10]:
import glob
import mido

midi_files_lista_limpiar = []

# Listar todos los archivos .mid en la carpeta "midis"
midi_files = glob.glob("midis/*.mid")

for midi_file in midi_files:
    midi = mido.MidiFile(midi_file)

    instrumentos_melodicos = set()
    canales_percusion = set()

    for track in midi.tracks:
        for msg in track:
            if msg.type == 'program_change':
                if msg.channel == 9:
                    canales_percusion.add(9)
                else:
                    instrumentos_melodicos.add((msg.channel, msg.program))

            elif msg.type in ['note_on', 'note_off']:
                if msg.channel == 9:
                    canales_percusion.add(9)

    n_instrumentos_melodicos = len(instrumentos_melodicos)
    n_canales_percusion = len(canales_percusion)

    #if n_instrumentos_melodicos > 8 or n_canales_percusion > 1:
    if (n_instrumentos_melodicos > 8 or n_canales_percusion > 1) and "modified" not in midi_file:
        print(f"Incompatible: '{midi_file}' → {n_instrumentos_melodicos} instrumentos melódicos, {n_canales_percusion} pista(s) de percusión")
        midi_files_lista_limpiar.append(midi_file)


Incompatible: 'midis/MC Mt. Crenel.mid' → 12 instrumentos melódicos, 1 pista(s) de percusión
Incompatible: 'midis/MC Hyrule Field (MT).mid' → 10 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/ST Unknown 5.mid' → 16 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/FS Title Theme.mid' → 9 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/MC Hyrule Town.mid' → 12 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/MC Staff Roll.mid' → 63 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/FS Staff Roll.mid' → 10 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/MC Cloud Tops.mid' → 35 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/ALTTP Dark World.mid' → 13 instrumentos melódicos, 0 pista(s) de percusión


##Núm de instrumentos usados en no aptos

Debería salir lista vacía si no se usan más de 8 melódicos y 1 drums

In [11]:
import glob
import mido

# Listar todos los archivos .mid en la carpeta "midis"
midi_files = glob.glob("midis/*.mid")

for midi_file in midi_files_lista_limpiar:
    midi = mido.MidiFile(midi_file)

    # Mapas para detectar si hay notas tocadas por cada canal
    canales_con_program_change = {}
    canales_con_notas = set()
    hay_percusion = False

    for track in midi.tracks:
        current_program = {}  # canal -> programa actual
        for msg in track:
            if msg.type == 'program_change':
                canales_con_program_change[msg.channel] = msg.program

            elif msg.type == 'note_on' and msg.velocity > 0:
                canal = msg.channel
                if canal == 9:
                    hay_percusion = True
                else:
                    canales_con_notas.add(canal)

    # Asociar solo los instrumentos que realmente tocaron notas
    instrumentos_usados = set()
    for canal in canales_con_notas:
        if canal in canales_con_program_change:
            programa = canales_con_program_change[canal]
            instrumentos_usados.add((canal, programa))
        else:
            # Si no hay program_change, asumimos instrumento por defecto (programa 0)
            instrumentos_usados.add((canal, 0))

    n_instrumentos_melodicos = len(instrumentos_usados)
    n_canales_percusion = 1 if hay_percusion else 0

    if (n_instrumentos_melodicos > 8 or n_canales_percusion > 1) and "modified" not in midi_file:
        print(f"Incompatible: '{midi_file}' → {n_instrumentos_melodicos} instrumentos melódicos, {n_canales_percusion} pista(s) de percusión")


Incompatible: 'midis/MC Hyrule Field (MT).mid' → 9 instrumentos melódicos, 0 pista(s) de percusión
Incompatible: 'midis/ST Unknown 5.mid' → 10 instrumentos melódicos, 0 pista(s) de percusión


#Eliminar instrumentos no utilizados

In [14]:
import glob
import mido
from mido import MidiFile, MidiTrack, Message
import os

# Directorio de entrada y salida
midi_files = glob.glob("midis/*.mid") # Iterar sobre todos los archivos si es necesario
output_dir = "midis_limpios/"

os.makedirs(output_dir, exist_ok=True)

for midi_file in midi_files_lista_limpiar:
    midi = MidiFile(midi_file)
    new_midi = MidiFile()
    new_midi.ticks_per_beat = midi.ticks_per_beat

    for i, track in enumerate(midi.tracks):
        # Identificar los canales que tienen notas en esta pista
        canales_con_notas_en_pista = set()
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0:
                canales_con_notas_en_pista.add(msg.channel)

        # Crear una nueva pista para los mensajes de este track original
        new_track_for_this_original = MidiTrack()

        for msg in track:
            # Mantener mensajes meta en su pista original o copiarlos
            if msg.is_meta:
                 new_track_for_this_original.append(msg)
            # Mantener mensajes de control, program change y notas solo para canales con notas o canal 9
            elif msg.type in ['program_change', 'control_change', 'note_on', 'note_off']:
                 if msg.channel in canales_con_notas_en_pista or msg.channel == 9: # Mantener drums (canal 9)
                     new_track_for_this_original.append(msg)
            # Puedes añadir otros tipos de mensajes si son relevantes para conservar

        # Añadir la nueva pista si no está vacía (excepto si solo contiene mensajes meta al inicio)
        # Una pista vacía o con solo mensajes meta iniciales no es útil
        if any(not m.is_meta for m in new_track_for_this_original):
             new_midi.tracks.append(new_track_for_this_original)
        elif all(m.is_meta for m in new_track_for_this_original) and new_track_for_this_original:
             # Si solo hay mensajes meta, añadir esta pista solo si es la primera pista original
             # para evitar duplicar metadatos como el tempo
             if i == 0:
                 new_midi.tracks.append(new_track_for_this_original)


    # Guardar el nuevo archivo solo si la nueva_midi tiene pistas (excluyendo una posible pista meta vacía)
    if any(not track for track in new_midi.tracks) or not new_midi.tracks:
        print(f"El archivo '{os.path.basename(midi_file)}' resultó vacío después de la limpieza.")
    else:
        output_path = os.path.join(output_dir, os.path.basename(midi_file))
        new_midi.save(output_path)
        print(f"Limpio: {os.path.basename(midi_file)} → guardado en {output_dir}")

Limpio: MC Mt. Crenel.mid → guardado en midis_limpios/
Limpio: MC Hyrule Field (MT).mid → guardado en midis_limpios/
Limpio: ST Unknown 5.mid → guardado en midis_limpios/
Limpio: FS Title Theme.mid → guardado en midis_limpios/
Limpio: MC Hyrule Town.mid → guardado en midis_limpios/
Limpio: MC Staff Roll.mid → guardado en midis_limpios/
Limpio: FS Staff Roll.mid → guardado en midis_limpios/
Limpio: MC Cloud Tops.mid → guardado en midis_limpios/
Limpio: ALTTP Dark World.mid → guardado en midis_limpios/


In [13]:
# borrar "midis_limpios"
!rm -rf midis_limpios/*